## Raghav Virmani - 101803657 - COE1 - ML Assignment 2

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn import preprocessing

In [ ]:
df = pd.read_csv('../input/microsoft-adventure-works-cycles-customer-data/AWCustomers.csv')
df_2 = pd.read_csv('../input/microsoft-adventure-works-cycles-customer-data/AWSales.csv') #Bike Buyer is our target column
df = df.merge(df_2)
df.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated,BikeBuyer,AvgMonthSpend
0,21173,NaN,Chad,C,Yuan,NaN,7090 C. Mount Hood,NaN,Wollongong,New South Wales,...,M,M,1,3,0,1,81916,2017-03-06,1,50.97
1,13249,NaN,Ryan,NaN,Perry,NaN,3651 Willow Lake Rd,NaN,Shawnee,British Columbia,...,M,M,1,2,1,2,81076,2017-03-06,1,53.11
2,29350,NaN,Julia,NaN,Thompson,NaN,1774 Tice Valley Blvd.,NaN,West Covina,California,...,F,S,0,3,0,0,86387,2017-03-06,1,54.08
3,13503,NaN,Theodore,NaN,Gomez,NaN,2103 Baldwin Dr,NaN,Liverpool,England,...,M,M,1,2,1,2,61481,2017-03-06,1,56.93
4,22803,NaN,Marshall,J,Shan,NaN,Am Gallberg 234,NaN,Werne,Nordrhein-Westfalen,...,M,S,1,1,0,0,51804,2017-03-06,1,55.41


Part 1a:
Feature Selection: Customer ID, Title, Name columns, suffix, Last Updated are unnecessary columns.  
Since there are 15k unique cities, address lines also won't help the model much.  
We will keep CountryRegionName column since there are limited unique values in it, this feature can be useful.  
Since we have Country, we don't need StateProvinceName or City.  
Number of children at home are enough we don't need total no. of children either.

In [ ]:
#1b
df = df.drop(columns=['CustomerID','Title','FirstName','MiddleName','PhoneNumber','LastName','Suffix','AddressLine1','AddressLine2','StateProvinceName','City','PostalCode','LastUpdated','TotalChildren'])
df.head()

,CountryRegionName,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,YearlyIncome,BikeBuyer,AvgMonthSpend
0,Australia,1987-11-13,Bachelors,Clerical,M,M,1,3,0,81916,1,50.97
1,Canada,1972-07-21,Partial College,Clerical,M,M,1,2,1,81076,1,53.11
2,United States,1985-11-09,Bachelors,Clerical,F,S,0,3,0,86387,1,54.08
3,United Kingdom,1977-10-18,Partial College,Skilled Manual,M,M,1,2,1,61481,1,56.93
4,Germany,1975-02-05,Partial College,Skilled Manual,M,S,1,1,0,51804,1,55.41


In [ ]:
#convert BirthDate into age
for i, row in df.iterrows():
    age = 2021 - int(df.loc[i,'BirthDate'][:4])
    df.at[i,'BirthDate'] = age
df.rename(columns={"BirthDate": "Age","CountryRegionName": "Country"},inplace=True)

In [ ]:
df.nunique(axis=0)

Country                     6
Age                        70
Education                   5
Occupation                  5
Gender                      2
MaritalStatus               2
HomeOwnerFlag               2
NumberCarsOwned             6
NumberChildrenAtHome        4
YearlyIncome            15356
BikeBuyer                   2
AvgMonthSpend            1803
dtype: int64

Part 1C:  
1. Country: Categorical Nominal  
2. Age: Continous and ratio  
3. YearlyIncome: Continous and ratio  
4. Occupation, Gender, MaritalStatus: Categorical Nominal  
5. HomeOwnerFlag, NumberCarsOwned, NumberChildrenAtHome, Education: Categorical Ordinal

In [ ]:
#Part 2A
df.isnull().any().sum().sum()
#Hence we have no null values to remove

0

In [ ]:
dfage = df[['Age']] #saving for later use

In [ ]:
#Part 2B we only need to do normalisation of age and yearly income
min_max_scaler = preprocessing.MinMaxScaler()
scaled_age = min_max_scaler.fit_transform(df[['Age']].values)
scaled_income = min_max_scaler.fit_transform(df[['YearlyIncome']].values)
scaled_spend = min_max_scaler.fit_transform(df[['AvgMonthSpend']].values)
df['Age_Scaled'] = scaled_age
df['YearlyIncome_Scaled'] = scaled_income
df['AvgMonthSpend_Scaled'] = scaled_spend

In [ ]:
#Part 2C, If we perform binning on continous data it is unnecessary throwing away of data,
#and we have only 1 nominal feature which has 5 values hence one-hot-encoding would not cause dimentionality problem

#Part 2E
df['Gender_Dummy'] = pd.get_dummies(df.Gender,drop_first=True)
df['MaritalStatus_Dummy'] = pd.get_dummies(df.MaritalStatus,drop_first=True)
df2 = pd.get_dummies(df.Country,drop_first=True)
df3 = pd.get_dummies(df.Occupation,drop_first=True)
df = pd.concat([df, df2, df3], axis=1)

In [ ]:
#Doing label encoding manually for education feature:
edu_dict = {'Partial High School':0,'High School':1,'Partial College':2,'Bachelors':3,'Graduate Degree':4}
for i, row in df.iterrows():
    ed = df.loc[i,'Education']
    df.at[i,'Education'] = edu_dict[ed]

In [ ]:
#Using Sklearn
# le = preprocessing.LabelEncoder()
# le.fit(df['Education'])
# le.transform(df['Education'])

In [ ]:
#Dropping the 
df.drop(columns = ['Gender','Country','Age','Occupation','MaritalStatus','YearlyIncome','AvgMonthSpend'],inplace=True)
df

,Education,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,BikeBuyer,Age_Scaled,YearlyIncome_Scaled,AvgMonthSpend_Scaled,Gender_Dummy,MaritalStatus_Dummy,Canada,France,Germany,United Kingdom,United States,Management,Manual,Professional,Skilled Manual
0,3,1,3,0,1,0.185714,0.496842,0.324210,1,0,0,0,0,0,0,0,0,0,0
1,2,1,2,1,1,0.400000,0.489453,0.425201,1,0,1,0,0,0,0,0,0,0,0
2,3,0,3,0,1,0.214286,0.536172,0.470977,0,1,0,0,0,0,1,0,0,0,0
3,2,1,2,1,1,0.328571,0.317083,0.605474,1,0,0,0,0,1,0,0,0,0,1
4,2,1,1,0,1,0.357143,0.231958,0.533742,1,1,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18356,4,0,1,0,0,0.142857,0.242065,0.190656,0,0,0,0,0,0,1,0,0,0,1
18357,3,0,2,0,1,0.114286,0.312781,0.289287,0,1,0,0,0,0,0,0,0,0,1
18358,2,0,2,0,1,0.242857,0.232442,0.257669,0,1,0,0,0,0,1,0,0,0,1
18359,1,0,0,0,0,0.071429,0.543121,0.345446,0,1,1,0,0,0,0,0,0,0,0


In [ ]:
def SMC(x,y):
    x,y = x.values, y.values
    return np.sum(x==y)/len(x)

In [ ]:
#Calculate similarity between HomeOwnerFlag and Gender
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import cosine
print('Simple Matching Coefficient:',SMC(df['HomeOwnerFlag'], df['Gender_Dummy']))
print('Jaccard Similarity Score:',jaccard_score(df['HomeOwnerFlag'], df['Gender_Dummy']))
print('Cosine Similarity Score:',1-cosine(df['HomeOwnerFlag'], df['Gender_Dummy']))

Simple Matching Coefficient: 0.5061271172594085
Jaccard Similarity Score: 0.3865927078400866
Cosine Similarity Score: 0.5600880873768459


In [ ]:
#Part 3: Since CommuteDistance feature is missing, we will calculate correlation between Age and YearlyIncome
df2 = df[['AvgMonthSpend_Scaled','YearlyIncome_Scaled']]
df2.corr(method='pearson')
#Therefore there is high positive correlation between Average Monthly Spend and Yearly Income

,AvgMonthSpend_Scaled,YearlyIncome_Scaled
AvgMonthSpend_Scaled,1.000000,0.530126
YearlyIncome_Scaled,0.530126,1.000000
